In [1]:
import numpy as np
import os
import shutil
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
def download_and_read(url):
    local_file = url.split('/')[-1]
    local_file = local_file.replace("%20", " ")
    p = tf.keras.utils.get_file(local_file, url,
        extract=True, cache_dir=".")
    local_folder = os.path.join("datasets", local_file.split('.')[0])
    labeled_sentences = []
    for labeled_filename in os.listdir(local_folder):
        if labeled_filename.endswith("_labelled.txt"):
            with open(os.path.join(
                    local_folder, labeled_filename), "r") as f:
                for line in f:
                    sentence, label = line.strip().split('\t')
                    labeled_sentences.append((sentence, label))
    return labeled_sentences
labeled_sentences = download_and_read(      
    "https://archive.ics.uci.edu/ml/machine-learning-databases/" + 
    "00331/sentiment%20labelled%20sentences.zip")
sentences = [s for (s, l) in labeled_sentences]
labels = [int(l) for (s, l) in labeled_sentences]

  40960/Unknown - 0s 4us/step

In [5]:
# tokenize sentences
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_counts)
print("vocabulary size: {:d}".format(vocab_size))

word2idx = tokenizer.word_index
idx2word = {v:k for (k, v) in word2idx.items()}

vocabulary size: 5271


In [8]:
seq_lengths = np.array([len(s.split()) for s in sentences])
print([(p, np.percentile(seq_lengths, p)) for p
    in [75, 80, 90, 95, 99, 100]])

[(75, 16.0), (80, 18.0), (90, 22.0), (95, 26.0), (99, 36.0), (100, 71.0)]


In [9]:
max_seqlen = 64
# create dataset
sentences_as_ints = tokenizer.texts_to_sequences(sentences)
sentences_as_ints = tf.keras.preprocessing.sequence.pad_sequences(
    sentences_as_ints, maxlen=max_seqlen)
labels_as_ints = np.array(labels)
dataset = tf.data.Dataset.from_tensor_slices(
    (sentences_as_ints, labels_as_ints))

In [10]:
dataset = dataset.shuffle(10000)
test_size = len(sentences) // 3
val_size = (len(sentences) - test_size) // 10
test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)
batch_size = 64
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [11]:
class SentimentAnalysisModel(tf.keras.Model):
    def __init__(self, vocab_size, max_seqlen, **kwargs):
        super(SentimentAnalysisModel, self).__init__(**kwargs)
        self.embedding = tf.keras.layers.Embedding(
            vocab_size, max_seqlen)
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(max_seqlen)
        )
        self.dense = tf.keras.layers.Dense(64, activation="relu")
        self.out = tf.keras.layers.Dense(1, activation="sigmoid")
    def call(self, x):
        x = self.embedding(x)
        x = self.bilstm(x)
        x = self.dense(x)
        x = self.out(x)
        return x

In [12]:
model = SentimentAnalysisModel(vocab_size+1, max_seqlen)
model.build(input_shape=(batch_size, max_seqlen))
model.summary()
# compile
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

Model: "sentiment_analysis_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  337408    
                                                                 
 bidirectional (Bidirectiona  multiple                 66048     
 l)                                                              
                                                                 
 dense (Dense)               multiple                  8256      
                                                                 
 dense_1 (Dense)             multiple                  65        
                                                                 
Total params: 411,777
Trainable params: 411,777
Non-trainable params: 0
_________________________________________________________________


In [13]:
data_dir = "./data"
logs_dir = os.path.join("./logs")
best_model_file = os.path.join(data_dir, "best_model.h5")
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model_file,
    save_weights_only=True,
    save_best_only=True)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)
num_epochs = 10
history = model.fit(train_dataset, epochs=num_epochs,
    validation_data=val_dataset,
    callbacks=[checkpoint, tensorboard])

Epoch 1/10
29/29 [==============================] - 7s 46ms/step - loss: 0.6910 - accuracy: 0.5417 - val_loss: 0.6866 - val_accuracy: 0.4950
Epoch 2/10
29/29 [==============================] - 0s 17ms/step - loss: 0.6231 - accuracy: 0.7022 - val_loss: 0.4042 - val_accuracy: 0.8550
Epoch 3/10
29/29 [==============================] - 0s 17ms/step - loss: 0.4163 - accuracy: 0.8317 - val_loss: 0.3112 - val_accuracy: 0.8950
Epoch 4/10
29/29 [==============================] - 1s 17ms/step - loss: 0.2362 - accuracy: 0.9161 - val_loss: 0.1851 - val_accuracy: 0.9450
Epoch 5/10
29/29 [==============================] - 1s 18ms/step - loss: 0.1730 - accuracy: 0.9433 - val_loss: 0.1284 - val_accuracy: 0.9450
Epoch 6/10
29/29 [==============================] - 0s 17ms/step - loss: 0.1113 - accuracy: 0.9644 - val_loss: 0.0386 - val_accuracy: 0.9950
Epoch 7/10
29/29 [==============================] - 1s 17ms/step - loss: 0.0672 - accuracy: 0.9811 - val_loss: 0.0536 - val_accuracy: 0.9850
Epoch 8/10
29

In [14]:
best_model = SentimentAnalysisModel(vocab_size+1, max_seqlen)
best_model.build(input_shape=(batch_size, max_seqlen))
best_model.load_weights(best_model_file)
best_model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [15]:
test_loss, test_acc = best_model.evaluate(test_dataset)
print("test loss: {:.3f}, test accuracy: {:.3f}".format(
    test_loss, test_acc))

16/16 [==============================] - 1s 12ms/step - loss: 0.0267 - accuracy: 0.9940
test loss: 0.027, test accuracy: 0.994


In [16]:
labels, predictions = [], []
idx2word[0] = "PAD"
is_first_batch = True
for test_batch in test_dataset:
   inputs_b, labels_b = test_batch
   pred_batch = best_model.predict(inputs_b)
   predictions.extend([(1 if p > 0.5 else 0) for p in pred_batch])
   labels.extend([l for l in labels_b])
   if is_first_batch:
       # print first batch of label, prediction, and sentence
       for rid in range(inputs_b.shape[0]):
           words = [idx2word[idx] for idx in inputs_b[rid].numpy()]
           words = [w for w in words if w != "PAD"]
           sentence = " ".join(words)
           print("{:d}\t{:d}\t{:s}".format(
               labels[rid], predictions[rid], sentence))
       is_first_batch = False
print("accuracy score: {:.3f}".format(accuracy_score(labels, predictions)))
print("confusion matrix")
print(confusion_matrix(labels, predictions))

0	0	soyo technology sucks
0	0	will not be back
1	1	is pretty funny
1	1	blue ant is easy to use
1	1	i can assure you that you won't be disappointed
1	1	lovely little thriller from hitchcock with lots of nice shenanigans surrounding a murdered spy a kidnapped child a nasty church a foreign plot and some random taxidermists
1	1	if there was ever a movie that needed word of mouth to promote this is it
0	0	i felt asleep the first time i watched it so i can recommend it for insomniacs
1	1	it's hard not to fall head over heels in love with that girl
0	0	only like 3 or 4 buildings used a couple of locations maybe poor hummh
0	0	and the rest of it just sits there being awful with soldiers singing songs about the masculinity they pledge themselves to hairsplitting about purity the admiration of swords etc
0	0	the commercials are the most misleading
0	0	it's too bad that everyone else involved didn't share crowe's level of dedication to quality for if they did we'd have a far better film on our h